# Lab 6

Here are the  classes from lecture 16. For this lab, you will use these classes for this lab and are encouraged to modify these classes as you need.

In [3]:
# Create some virtual classes

class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name
    

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [4]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print self.name()+" Error: Bad Grade."
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    __keys=list()
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        self.__keys=list()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
            self.__keys.append(a_grade.gradebook_name())
        else:
            print self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists."
            raise Exception

    def id_number(self):
        return __id_number
    
    def __getitem__(self,key):
        
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print self.__grades[grade]
    
    def remove(self,key):
        del self.__grades[key]
         
        for i in range(0,len(self.__keys)-1):
             
            if self.__keys[i]==key:
                del self.__keys[i]
    def get_keys(self):
        return self.__keys


In [5]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


In [6]:
 class grade_book(data):
    __students=dict()
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
    
    def add_student(self,a_student,number):
        
        self.__students[number]=a_student
    def __getitem__(self,key):

        return  self.__students[key]
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print self.name()+" Error: Did not find student."
            
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.iteritems():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.iteritems():
            grades.append(a_student[grade_name].value())
            
        return a_stat_comp.apply(grades)
        
    def print_it(self,key):
        print self.__students[1].__getitem__(key) 
    def get_students(self):
        return  self.__students
     

In [7]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().iteritems():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print self.name()+ " Error: Did not get a numerical grade as input."
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                #print percent, i, v
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [8]:
import numpy as np
import math


class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().iteritems():
            grades.append(float(a_student[grade_name].value()))
            
        for k,a_student in a_grade_book.get_students().iteritems():
            a_student.add_grade(grade(grade_name+" Mean",value=np.mean(grades)))
            a_student.add_grade(grade(grade_name+" STD",value=math.sqrt(np.var(grades))))
          


## CSV Reader

*Exercise 1*: The data for *your* grades in this course are stored (anonymized) in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

In [9]:
!cat Data1401-Grades.csv 

l1_n,l1_1,l2_n,l2_1,l2_2,l2_3,l2_4,l2_5,l2_6,l2_7,l3_n,l3_1,l3_2,l3_3,l3_4,l3_5,l3_6,l3_7,l3_8,l3_9,l3_10,l3_11,l3_12,l3_13,l3_14,l4_n,l4_1,l4_2,l4_3,l4_4,l4_5,l4_6,l4_7,l4_8,l4_9,l4_10,l4_11,q1_n,q1_1,e1_n,e1_1,e1_2,e1_3,e1_4,e1_5,e1_6,e1_7,e1_8,e1_9,e1_10,e1_11,e1_12,e1_13,e1_14,e1_15
1,10,7,0,10,10,8,10,10,10,14,9,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,9.5,15,9,9,0,9,8,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,7,0,0,0,0,0,0,0,14,9,10,10,10,7,10,3,6,3,3,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,5,15,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0
1,10,7,10,10,3,9.5,10,10,9.5,14,10,10,10,8,5,10,5,10,3,0,10,3,10,8,11,10,10,10,10,10,10,0,0,10,5,0,1,10,15,9,9,10,9,7,9,0,0,10,10,9,5,10,8,10
1,10,7,10,10,9.5,0,10,10,0,14,9.5,0,0,10,0,10,5,10,7,0,10,6,10,0,11,10,10,6,0,0,0,0,0,0,0,0,1,0,15,0,0,0,0,5,0,7,0,3,3,3,0,3,0,0
1,10,7,10,10,10,9.5,10,10,9.5,14,5,9.5,9.5,8,10,10,8,10,8,0,5,6,0,0,11,0,10,10,10,0,5,0,

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture 7 we created a simple CSV file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

In [10]:
def cvs_reader(filename):
    data=list() 
    a = open(filename).read().splitlines()
    hold=list()
    for i in range(0,len(a)):
        hold.append(a[i].split(","))

    for i in range(0,len(hold)):
        try:
            data.append(dict(zip(hold[0],hold[i+1])))
        except(IndexError):
            break;
    return data


Data=cvs_reader("Data1401-Grades.csv")

## Creating a Gradebook

*Exercise 2:* In lectures 15 and 16 we used pandas to read the CSV file and create the grade book. Modify the code below to use your CSV reader instead.

In [11]:
a_grade_book=grade_book("Data 1401")
 
for student_i in range(len(Data)):
     
    a_student_0=student("Student",str(student_i),student_i)
     
    for k in Data[0].keys():
         
        a_student_0.add_grade(grade(k,value=(float(Data[student_i][k]))))
     
    a_grade_book.add_student(a_student_0,student_i)

 
 
 

## Grade Summing

*Exercise 3:* In lecture 16 we changed the design of our algorithms classes and then updated the `uncurved_letter_grade_percent` calculator. In lecture 15 we create a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each students.



In [12]:
 
class grade_sum:
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n+1
         
        
    def apply(self,a_student):
        labels=[self.__prefix+str(x) for x in range(1,self.__n)]
        
        grade_sum=0.
        for label in labels:
            a= str(a_student.__getitem__(label))
            b= a.split(":")
            number= float(b[1])
             
            grade_sum=number+grade_sum
        print grade_sum 
        a_student.add_grade(grade(self.__prefix+"sum",value=grade_sum))
        
print 'test1'
ts= grade_sum('e1_',int(Data[0]['e1_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    ts.apply(a)
# For testing if add the e1_sum correctly
# a_grade_book.__getitem__(3).print_grades() 
print 'quizz1'
qs= grade_sum('q1_',int(Data[0]['q1_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    qs.apply(a)
    
print 'lab1'
l1s= grade_sum('l1_',int(Data[0]['l1_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    l1s.apply(a)
    
print 'lab2'
l2s= grade_sum('l2_',int(Data[0]['l2_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    l2s.apply(a) 
    
print 'lab3'
l3s= grade_sum('l3_',int(Data[0]['l3_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    l3s.apply(a)
    
print 'lab4'
l4s= grade_sum('l4_',int(Data[0]['l4_n']))
for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    l4s.apply(a)
    
 

test1
35.0
0.0
20.0
115.0
24.0
95.0
37.0
77.0
103.0
117.0
138.0
110.0
104.0
111.0
103.0
134.0
quizz1
9.5
0.0
5.0
10.0
0.0
9.5
10.0
10.0
9.5
9.5
0.0
10.0
10.0
0.0
0.0
9.5
lab1
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
10.0
lab2
58.0
0.0
0.0
62.0
49.5
69.0
54.5
69.0
39.5
60.0
47.0
68.5
64.0
49.5
68.0
59.0
lab3
9.0
0.0
71.0
102.0
77.5
89.0
64.5
40.0
40.0
26.0
126.0
97.5
115.0
114.5
95.0
106.0
lab4
0.0
0.0
0.0
75.0
26.0
35.0
30.0
51.0
68.0
27.0
105.0
51.0
0.0
25.0
0.0
53.0


## Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [13]:
mtc=mean_std_calculator()
mtc.apply(a_grade_book,'e1_sum')
mtc.apply(a_grade_book,'l1_sum')
mtc.apply(a_grade_book,'l2_sum')
mtc.apply(a_grade_book,'l3_sum')
mtc.apply(a_grade_book,'l4_sum')
mtc.apply(a_grade_book,'q1_sum')
print a_grade_book.__getitem__(2).__getitem__('e1_sum Mean')
# add function to grade_book
a_grade_book.print_it(('e1_sum Mean'))
a_grade_book.print_it(('e1_sum STD'))
a_grade_book.print_it(('l1_sum Mean'))
a_grade_book.print_it(('l1_sum STD'))
a_grade_book.print_it(('l2_sum Mean'))
a_grade_book.print_it(('l2_sum STD'))
a_grade_book.print_it(('l3_sum Mean'))
a_grade_book.print_it(('l3_sum STD'))
a_grade_book.print_it(('l4_sum Mean'))
a_grade_book.print_it(('l4_sum STD'))
a_grade_book.print_it(('q1_sum Mean'))
a_grade_book.print_it(('q1_sum STD'))

e1_sum Mean: 82.6875
e1_sum Mean: 82.6875
e1_sum STD: 42.9370451213
l1_sum Mean: 10.0
l1_sum STD: 0
l2_sum Mean: 51.09375
l2_sum STD: 21.0566340125
l3_sum Mean: 73.3125
l3_sum STD: 38.3017929574
l4_sum Mean: 34.125
l4_sum STD: 30.4217746852
q1_sum Mean: 6.40625
q1_sum STD: 4.46940554632


*Exercise 5:* In lecture 16 we changed the design of our algorithms classes and then updated the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [14]:
class curved_letter_grade(calculator):
    
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        
        self.definition=[ (self.__mean+ self.__std,"A"),
                          (self.__mean,"B"),
                          (self.__mean- self.__std,"C"),
                          (self.__mean- self.__std*2,"D"),
                          (self.__mean- self.__std*3,"F"),]  
    def apply(self,grade):
        for i in range(0,5):
            
            if grade>=self.definition[i][0]:
                return   self.definition[i][1]
print 'exam1 letter grade'    
clg=curved_letter_grade('exam1',82.6875,42.9370451213,max_grade=150)
for i in range (len(Data)): 
    a=str( a_grade_book.__getitem__(i).__getitem__('e1_sum') )
     
    b= a.split(":")
    print clg.apply(float(b[1]))

print 'lab1 letter grade'    
clg=curved_letter_grade('lab1',10.0,0.0,max_grade=10.0)
for i in range (len(Data)): 
    a=str( a_grade_book.__getitem__(i).__getitem__('l1_sum') )
     
    b= a.split(":")
    print clg.apply(float(b[1]))


 

exam1 letter grade
D
D
D
B
D
B
D
C
B
B
A
B
B
B
B
A
lab1 letter grade
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A


## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [15]:
 
class grade_drop:
    def __init__(self,n):
        self.__prefix ='l'
        self.__n=n+1
         
        
    def apply(self,a_student):
        labels=[]
        for i in range (1,self.__n):
            labels.append(self.__prefix+str(i)+'_sum') 
            hold=a_student.__getitem__(self.__prefix+str(i)+'_n')
            hold1=str(hold)
            hold2=hold1.split(":")
                     
            max_grade=float(hold2[1])*10.0
        grade=list()
        percent=list()
        for label in labels:
            a= str(a_student.__getitem__(label))
            b= a.split(":")
            grade.append(float(b[1]))
            c=float(b[1])/max_grade
            percent.append(c) 
         
        for i in range(self.__n):
    
            if min(percent)==percent[i]:
                a_student.remove(labels[i])
                print labels[i]+' get dropped'
                break
#Code for implement the drop_grade class
drop= grade_drop(4)

for i in  range(len(Data)):
    a=a_grade_book.__getitem__(i)
    print 'student '+str(i)
    drop.apply(a)
    
#testing if drop successfully in grade book
print 'student 1 grade book'
a_grade_book.__getitem__(1).print_grades()

student 0
l4_sum get dropped
student 1
l2_sum get dropped
student 2
l2_sum get dropped
student 3
l1_sum get dropped
student 4
l1_sum get dropped
student 5
l1_sum get dropped
student 6
l1_sum get dropped
student 7
l1_sum get dropped
student 8
l1_sum get dropped
student 9
l1_sum get dropped
student 10
l1_sum get dropped
student 11
l1_sum get dropped
student 12
l4_sum get dropped
student 13
l1_sum get dropped
student 14
l4_sum get dropped
student 15
l1_sum get dropped
student 1 grade book
q1_n: 1.0
l3_n: 14.0
l4_n: 11.0
l4_sum STD: 30.4217746852
e1_15: 0
e1_14: 0
e1_13: 0
e1_12: 0
e1_11: 0
e1_10: 0
l3_sum Mean: 73.3125
l3_2: 0
l2_sum Mean: 51.09375
q1_sum STD: 4.46940554632
l1_sum STD: 0
q1_sum: 0
l2_n: 7.0
l4_11: 0
l4_10: 0
l3_1: 0
l4_sum: 0
e1_sum: 0
l3_6: 0
l1_n: 1.0
l3_7: 0
l3_4: 0
e1_9: 0
e1_8: 0
q1_1: 0
e1_sum STD: 42.9370451213
e1_3: 0
e1_2: 0
e1_1: 0
l3_5: 0
e1_7: 0
e1_6: 0
e1_5: 0
e1_4: 0
l2_3: 0
l2_2: 0
l2_1: 0
l2_7: 0
l2_6: 0
l2_5: 0
l2_4: 0
e1_sum Mean: 82.6875
l3_sum: 0
l4_9:

*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [16]:
 
class lettergrade:
     
    def __init__(self):
          self.GradeMap=[ (12,"A+"),
                          (11,"A"),
                          (10,"A-"),
                          (9,"B+"),
                          (8,"B"),
                            (7,"B-"),(6,"C+"),(5,"C"),(4,"C-"),(3,"D+"),(2,"D"),(1,"D-"),(0,"F"),]
            
           
         
    def apply(self,a_student,n=4):
        labels=['e1_sum','q1_sum','l1_sum','l2_sum','l3_sum','l4_sum']
         
       
        labe=['e1_n','q1_n','l1_n','l2_n','l3_n','l4_n']
        for i in range (0,6):
            
            if labels[i] not in a_student.get_keys():
                del labels[i]
                del labe[i]
                break
        max_grade=0

        for label in labe:
            a= str(a_student.__getitem__(label))
            b= a.split(":")
            number= float(b[1])*10.0
            max_grade=number+max_grade
        grade_sum=0.
        for label in labels:
            a= str(a_student.__getitem__(label))
            b= a.split(":")
            number= float(b[1])
            grade_sum=number+grade_sum
        
                          
        grade=grade_sum/max_grade*12.0
        
        for i in range(0,13):
            
            if grade>=self.GradeMap[i][0]:
                 
                return   self.GradeMap[i][1]
            
 

Test you calculator by applying the weights from the sy

In [17]:
letter=lettergrade()
for i in range(len(Data)):
    a=a_grade_book.__getitem__(i)
    print letter.apply(a)
        

D+
F
D+
B+
C-
B-
C-
C+
C+
C
A-
B
B+
B-
B
B+
